In [3]:
import pickle

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [4]:
with open('/home/arash/ProjectVR/cleaneddata/ALLDATA', 'rb') as f:
    data=pickle.load(f)
    
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'Vrexp_Yes']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later

data=pd.get_dummies(data=data,columns=['Speed Limit','Lane Width','Minimum Gap','Mean Arrival Rate','numcars'])



data=data.apply(pd.to_numeric, errors='coerce')


def FullAV(row):
    if row['AV']==100:
        return 1
    else:
        return 0
def MixedAV(row):
    if row['AV']>0 and row['AV']<100:
        return 1
    else:
        return 0   
def HDV(row):
    if row['AV']==0:
        return 1

    else:
        return 0   
data=data[data.Occupation_kid==0]                         #remove kids


data['FullAV']=data.apply (lambda row: FullAV(row), axis=1)
data['Mixed']=data.apply (lambda row: MixedAV(row), axis=1)
#data['HDV']=data.apply (lambda row: HDV(row), axis=1)
data['over50']=data['Age_50 - 59']+data['Age_60+']
data=data.drop(columns=['Age_9-12', 'Age_15-18', 'Age_12-15','Occupation_kid','AV','Age_50 - 59','Age_60+'])
data['E']=1




data=data.drop(columns=['Full Braking Before Impact_3', 'two way', 'Occupation_Unemployed',
                        'Education_Doctorate degree','mode_Bike','workwalk_Yes','shopwalk_Yes',
                        'Speed Limit_50.0','Lane Width_3.0','Minimum Gap_2.0','Mean Arrival Rate_1100.0'
                        ,'numcars_4'])


train, test = train_test_split(data,test_size=0.2, random_state=1)
             
#20 percent of data is seperated for test set
NCtrain=train   
NCtest= test
#NC: Not changed, used later for shap, without a format change that deepsurv requires

trainsub, valsub = train_test_split(train,test_size=0.2, random_state=1)  
#20 percent of train data is selected for validation set, 
#for cross-validation train is used,
#for network training using opt_params, trainsub and valsub are used

In [5]:
data.shape

(2291, 44)

In [6]:
def dataframe_to_deepsurv_ds(df, event_col = 'E', time_col = 'Wait Time (s)'):
    # Extract the event and time columns as numpy arrays
    e = df[event_col].values.astype(np.int32)
    t = df[time_col].values.astype(np.float32)

    # Extract the patient's covariates as a numpy array
    x_df = df.drop([event_col, time_col], axis = 1)
    x = x_df.values.astype(np.float32)
    
    # Return the deep surv dataframe
    return {
        'x' : x,
        'e' : e,
        't' : t
    }

In [7]:
train=dataframe_to_deepsurv_ds(train)
test=dataframe_to_deepsurv_ds(test)
trainsub = dataframe_to_deepsurv_ds(trainsub)
valsub = dataframe_to_deepsurv_ds(valsub)

In [8]:
xtrain = train['x']
etrain = train['e']
ttrain = train['t']
ytrain = np.column_stack((etrain, ttrain))

In [9]:
xtest = test['x']
etest = test['e']
ttest = test['t']
ytest = np.column_stack((etest, ttest))

In [10]:
xtrainsub = trainsub['x']
etrainsub = trainsub['e']
ttrainsub = trainsub['t']
ytrainsub = np.column_stack((etrainsub, ttrainsub))

In [11]:
xvalsub = valsub['x']
evalsub = valsub['e']
tvalsub = valsub['t']
yvalsub = np.column_stack((evalsub, tvalsub))

In [12]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'wb') as f:
    pickle.dump(xtrain,f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrain', 'wb') as f:
    pickle.dump(ytrain,f)  
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtest', 'wb') as f:
    pickle.dump(xtest,f)   

with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytest', 'wb') as f:
    pickle.dump(ytest,f)   

In [13]:

with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrainsub', 'wb') as f:
    pickle.dump(xtrainsub,f)   
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrainsub', 'wb') as f:
    pickle.dump(ytrainsub,f) 
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xvalsub', 'wb') as f:
    pickle.dump(xvalsub,f) 
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/yvalsub', 'wb') as f:
    pickle.dump(yvalsub,f) 

In [14]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'wb') as f:
    pickle.dump(NCtrain,f)  
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'wb') as f:
    pickle.dump(NCtest,f) 